In [1]:
%%time
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time, psycopg2, os, json
import pandas as pd
from itertools import chain
from sqlalchemy import create_engine
from datetime import datetime, date, timedelta
from collections import Counter

CPU times: total: 1.47 s
Wall time: 2.31 s


In [2]:
#Function to Interact with Database

def stats_loader(dataframe,table_name):
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
    }

    connection = psycopg2.connect(**db_params)

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

    dataframe.to_sql(table_name, engine, if_exists='append', index=False)

    connection.commit()
    connection.close()
    print('Push to database successfully')
    

    
def stats_reader():
    db_params = {
        'host': 'ec2-54-73-196-88.eu-west-1.compute.amazonaws.com',
        'database': 'd28e00sm4enoq3',
        'user': 'trrhxlkhqhzbte',
        'password': '9f83628fbaee04643a0ff70da7a7e38b6547fae9bfaaba98fc0a56abf39fade9',
        }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    query = f"SELECT * FROM pre_game_stats"
    cursor.execute(query)

    result = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    cursor.close()
    connection.close()

    # Create a DataFrame from the fetched data
    from_db = pd.DataFrame(result, columns=column_names)
    return from_db

## ML Data Extraction, Wrangling and Model Building

In [3]:
df = stats_reader()

In [4]:
df.head()

,Date,Time,Home_Team,Away_Team,match_links,league,Home_Matches,Home_Goals_scored,Home_Goals_conceded,Home_Assists,...,Away_Duels_won,Away_Ground_duels_won,Away_Aerial_duels_won,Away_Possession_lost,Away_throwIns,Away_goalKicks,Away_Offsides,Away_Fouls,Away_Yellow,Away_Red_cards
0,2023-10-06,-,Birmingham,West Brom,https://www.sofascore.com/birmingham-city-west...,Championship,10,12,10,9,...,51.2 (51.5%),31 (50.1%),20.2 (53.9%),145.7,27.1,7,1.6,7.8,1.8,0
1,2023-10-06,-,Athletic Club,Almería,https://www.sofascore.com/almeria-athletic-clu...,La Liga,8,13,9,9,...,54.9 (51%),39.6 (50.7%),15.3 (51.7%),124.5,19.1,7.3,1.3,13.6,2.3,1
2,2023-10-05,FT,FC Andorra,Real Zaragoza,https://www.sofascore.com/fc-andorra-real-zara...,La Liga2,9,9,14,5,...,48.7 (53.4%),33.6 (52.7%),15.1 (55.1%),125.9,19.4,8.2,2.4,11.2,2.1,1
3,2023-10-05,FT,Mirandés,Eibar,https://www.sofascore.com/mirandes-eibar/OgbsR...,La Liga2,9,12,16,6,...,48.6 (47%),31.7 (45.2%),16.9 (50.7%),153.4,29.7,5.6,2.7,13.9,2.6,1
4,2023-10-05,FT,Racing de Ferrol,Amorebieta,https://www.sofascore.com/sd-amorebieta-racing...,La Liga2,9,9,9,5,...,47.7 (45.8%),33.2 (46.5%),14.4 (44.2%),146.6,20,8.4,1.4,15.4,1.7,0


In [5]:
df.dtypes

Date              datetime64[ns]
Time                      object
Home_Team                 object
Away_Team                 object
match_links               object
                       ...      
Away_goalKicks            object
Away_Offsides             object
Away_Fouls                object
Away_Yellow               object
Away_Red_cards            object
Length: 106, dtype: object

In [6]:
df_2 = df.copy(deep=True)

In [7]:
#Wrangling Functions
def first_conversion(element):
    convert = element.split('/')
    if len(convert) == 2:
        if convert[1] == '0':
            convert = float('0')
        else:
            convert = float(convert[0])/float(convert[1])
    else:
        convert = float(convert[0])
    return convert

def second_conversion(element):
    convert = float((element.split('('))[1].replace(')','').replace('%',''))
    return convert

def third_conversion(element):
    convert = float(element.replace('%',''))
    return convert

In [8]:
#Wrangling scripts
for col in list(df_2.columns)[6:]:
    try:
        df_2[col] = df_2[col].astype(float)
    except:
        if '/' in df_2[col][0]:
            df_2[col] = df_2[col].apply(first_conversion)
        elif ('(' in df_2[col][0]) & ('%' in df_2[col][0]):
            df_2[col] = df_2[col].apply(second_conversion)
        elif ('%' in df_2[col][0]):
            df_2[col] = df_2[col].apply(third_conversion)
            
df_2.head()

,Date,Time,Home_Team,Away_Team,match_links,league,Home_Matches,Home_Goals_scored,Home_Goals_conceded,Home_Assists,...,Away_Duels_won,Away_Ground_duels_won,Away_Aerial_duels_won,Away_Possession_lost,Away_throwIns,Away_goalKicks,Away_Offsides,Away_Fouls,Away_Yellow,Away_Red_cards
0,2023-10-06,-,Birmingham,West Brom,https://www.sofascore.com/birmingham-city-west...,Championship,10.0,12.0,10.0,9.0,...,51.5,50.1,53.9,145.7,27.1,7.0,1.6,7.8,1.8,0.0
1,2023-10-06,-,Athletic Club,Almería,https://www.sofascore.com/almeria-athletic-clu...,La Liga,8.0,13.0,9.0,9.0,...,51.0,50.7,51.7,124.5,19.1,7.3,1.3,13.6,2.3,1.0
2,2023-10-05,FT,FC Andorra,Real Zaragoza,https://www.sofascore.com/fc-andorra-real-zara...,La Liga2,9.0,9.0,14.0,5.0,...,53.4,52.7,55.1,125.9,19.4,8.2,2.4,11.2,2.1,1.0
3,2023-10-05,FT,Mirandés,Eibar,https://www.sofascore.com/mirandes-eibar/OgbsR...,La Liga2,9.0,12.0,16.0,6.0,...,47.0,45.2,50.7,153.4,29.7,5.6,2.7,13.9,2.6,1.0
4,2023-10-05,FT,Racing de Ferrol,Amorebieta,https://www.sofascore.com/sd-amorebieta-racing...,La Liga2,9.0,9.0,9.0,5.0,...,45.8,46.5,44.2,146.6,20.0,8.4,1.4,15.4,1.7,0.0


In [9]:
df_2.dtypes

Date              datetime64[ns]
Time                      object
Home_Team                 object
Away_Team                 object
match_links               object
                       ...      
Away_goalKicks           float64
Away_Offsides            float64
Away_Fouls               float64
Away_Yellow              float64
Away_Red_cards           float64
Length: 106, dtype: object

#### Match Outcome Extraction

In [10]:
#Get scores
def get_scores(url):
    try:
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        options.add_argument(f'user-agent={userAgent}')
        options.add_argument('--blink-settings=imagesEnabled=false')
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--window-size=1920,1080")
        capa = DesiredCapabilities.CHROME
        capa["pageLoadStrategy"] = "none"

        driver = webdriver.Chrome(r'C:\Users\hp\Documents\Our Documents\Personal Development\Projects\Client Projects\Dexter Hadeveld (Upwork)\Development\ETL-Pipeline-Sport-Insight\app\chromedriver-win64\chromedriver.exe',options=options,desired_capabilities=capa)


        driver.get(url)
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//span[@class='sc-jEACwC cgJaFs']")))
        driver.execute_script("window.stop();")
        outcomes =driver.find_elements(By.CLASS_NAME, 'sc-fqkvVR.a-DkxQ')
        outcome = [con.get_attribute('innerText') for con in outcomes]
        data  = [item.replace('\n', ',') for item in outcome]
        data = (data[1].split(','))[1]
        #print(data)
        print('gotten')
        scores.append(data)
    except:
        print('Exception')
        scores.append('')

In [11]:
scores = []

for i in df_2.match_links:
    get_scores(i)

df_2['Outcome'] = scores

gotten
gotten
gotten
gotten
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
Exception
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
Exception
gotten
gotten
gotten
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
Exception
gotten
gotten
gotten
gotten
Exception
gotten
Exception
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
Exception
Exception
gotten
gotten
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
Exception
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten
gotten


C:\Users\hp\AppData\Local\Temp\ipykernel_32372\3654487979.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2['Outcome'] = scores


In [12]:
df_2.head()

,Date,Time,Home_Team,Away_Team,match_links,league,Home_Matches,Home_Goals_scored,Home_Goals_conceded,Home_Assists,...,Away_Ground_duels_won,Away_Aerial_duels_won,Away_Possession_lost,Away_throwIns,Away_goalKicks,Away_Offsides,Away_Fouls,Away_Yellow,Away_Red_cards,Outcome
0,2023-10-06,-,Birmingham,West Brom,https://www.sofascore.com/birmingham-city-west...,Championship,10.0,12.0,10.0,9.0,...,50.1,53.9,145.7,27.1,7.0,1.6,7.8,1.8,0.0,3 - 1
1,2023-10-06,-,Athletic Club,Almería,https://www.sofascore.com/almeria-athletic-clu...,La Liga,8.0,13.0,9.0,9.0,...,50.7,51.7,124.5,19.1,7.3,1.3,13.6,2.3,1.0,3 - 0
2,2023-10-05,FT,FC Andorra,Real Zaragoza,https://www.sofascore.com/fc-andorra-real-zara...,La Liga2,9.0,9.0,14.0,5.0,...,52.7,55.1,125.9,19.4,8.2,2.4,11.2,2.1,1.0,0 - 1
3,2023-10-05,FT,Mirandés,Eibar,https://www.sofascore.com/mirandes-eibar/OgbsR...,La Liga2,9.0,12.0,16.0,6.0,...,45.2,50.7,153.4,29.7,5.6,2.7,13.9,2.6,1.0,1 - 3
4,2023-10-05,FT,Racing de Ferrol,Amorebieta,https://www.sofascore.com/sd-amorebieta-racing...,La Liga2,9.0,9.0,9.0,5.0,...,46.5,44.2,146.6,20.0,8.4,1.4,15.4,1.7,0.0,


In [13]:
df_2 = df_2[df_2['Outcome'] != '']

#### Model Development

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
list_columns = [column for column in list(df_2.columns)]
list_columns = list_columns[6:-1]
#list_columns

In [16]:
x = df_2[list_columns]
x.head()

,Home_Matches,Home_Goals_scored,Home_Goals_conceded,Home_Assists,Home_Penalty_goals,Home_Free_kick_goals,Home_Goals_from_inside_the_box,Home_Goals_from_outside_the_box,Home_Left_foot_goals,Home_Right_foot_goals,...,Away_Duels_won,Away_Ground_duels_won,Away_Aerial_duels_won,Away_Possession_lost,Away_throwIns,Away_goalKicks,Away_Offsides,Away_Fouls,Away_Yellow,Away_Red_cards
0,10.0,12.0,10.0,9.0,0.5,0.0,0.120482,0.046512,2.0,10.0,...,51.5,50.1,53.9,145.7,27.1,7.0,1.6,7.8,1.8,0.0
1,8.0,13.0,9.0,9.0,1.0,0.0,0.154930,0.076923,4.0,7.0,...,51.0,50.7,51.7,124.5,19.1,7.3,1.3,13.6,2.3,1.0
2,9.0,9.0,14.0,5.0,1.0,0.0,0.093750,0.052632,5.0,4.0,...,53.4,52.7,55.1,125.9,19.4,8.2,2.4,11.2,2.1,1.0
3,9.0,12.0,16.0,6.0,1.0,0.5,0.160714,0.036364,1.0,10.0,...,47.0,45.2,50.7,153.4,29.7,5.6,2.7,13.9,2.6,1.0
5,9.0,6.0,14.0,4.0,1.0,0.0,0.090909,0.027027,3.0,3.0,...,53.8,53.4,54.7,134.0,19.1,7.7,1.9,14.8,2.2,0.0


In [17]:
Home_score = [float(score.replace(' ','').split('-')[0]) for score in list(df_2['Outcome'])]
Away_score = [float(score.replace(' ','').split('-')[1]) for score in list(df_2['Outcome'])]

In [18]:
Match_Outcome = []
Match_Outcome_Label = []
for score in list(df_2['Outcome']):
    score = score.replace(' ','').split('-')
    if score[0] > score[1]:
        Match_Outcome.append('Home_Win')
        Match_Outcome_Label.append(0)
    elif score[0] < score[1]:
        Match_Outcome.append('Home_Lose')
        Match_Outcome_Label.append(1)
    else:
        Match_Outcome.append('Home_Draw')
        Match_Outcome_Label.append(2)

In [53]:
Match_Outcome_Dict = {'0':'Home_Win','1':'Home_Lose','2':'Home_Draw'}

In [49]:
#Match_Outcome_Label

#### Feature Selection

In [47]:
from sklearn.linear_model import LinearRegression
from Selection_Method.Forward_Stepwise import forward_stepwise

In [48]:
%%time
#define your linear regression object
linear_model = LinearRegression()
mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(100, 200, 100, 50),  # Define the architecture of the neural network
    activation="relu",            # Activation function (e.g., "relu", "tanh")
    solver="adam",                # Optimization algorithm
    max_iter=2000,                # Maximum number of training iterations
    random_state=42               # Random seed for reproducibility
)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x[selected_features], Home_score, test_size=0.2, random_state=10)

# Standardize features (scaling)
scaler = StandardScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)


#initialize forward_stepwise object
selection = forward_stepwise(linear_model)
x_train_df = pd.DataFrame(x_train_scale, columns=list(x.columns))
x_test_df = pd.DataFrame(x_test_scale, columns=list(x.columns))

#input your train and test dataset into the .select_features() method and execute.
forward_list, final_score = selection.select_features(x_train_df, x_test_df, y_train, y_test)
print(forward_list, final_score)

['Away_Big_chances', 'Away_Errors_leading_to_shot', 'Home_freeKicks', 'Away_Clean_sheets', 'Away_Acc._opposition_half', 'Away_Saves_made', 'Away_Free_kick_goals', 'Home_Right_foot_goals', 'Away_Yellow', 'Away_Acc._crosses', 'Away_Left_foot_goals', 'Away_Errors_leading_to_goal', 'Home_Succ._dribbles', 'Away_Penalty_goals', 'Away_blockedScoringAttempt', 'Away_Fouls', 'Away_Clearance_off_line', 'Away_Duels_won', 'Home_Errors_leading_to_goal', 'Away_Succ._dribbles', 'Home_goalKicks', 'Home_Acc._long_balls', 'Home_Clearances_per_game', 'Away_fastBreaks', 'Away_penaltiesCommited', 'Home_Hit_woodwork', 'Home_Big_chances_missed', 'Home_penaltiesCommited', 'Away_Goals_conceded', 'Away_ballRecovery', 'Away_Assists'] 0.936541058890205
CPU times: total: 26.6 s
Wall time: 13.4 s


In [49]:
selected_features = ['Away_Big_chances', 'Away_Errors_leading_to_shot', 'Home_freeKicks', 'Away_Clean_sheets', 'Away_Acc._opposition_half', 'Away_Saves_made', 'Away_Free_kick_goals', 'Home_Right_foot_goals', 'Away_Yellow', 'Away_Acc._crosses', 'Away_Left_foot_goals', 'Away_Errors_leading_to_goal', 'Home_Succ._dribbles', 'Away_Penalty_goals', 'Away_blockedScoringAttempt', 'Away_Fouls', 'Away_Clearance_off_line', 'Away_Duels_won', 'Home_Errors_leading_to_goal', 'Away_Succ._dribbles', 'Home_goalKicks', 'Home_Acc._long_balls', 'Home_Clearances_per_game', 'Away_fastBreaks', 'Away_penaltiesCommited', 'Home_Hit_woodwork', 'Home_Big_chances_missed', 'Home_penaltiesCommited', 'Away_Goals_conceded', 'Away_ballRecovery', 'Away_Assists']

#### Home ML Model

In [82]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x[selected_features], Home_score, test_size=0.2, random_state=10)

# Standardize features (scaling)
scaler = StandardScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

In [83]:
%%time
# Step 3: Create and train the MLPRegressor model
#['identity', 'logistic', 'relu', 'softmax', 'tanh']

mlp_regressor_1 = MLPRegressor(
    hidden_layer_sizes=(100, 200, 100, 50),  # Define the architecture of the neural network
    activation="relu",            # Activation function (e.g., "relu", "tanh")
    solver="adam",                # Optimization algorithm
    max_iter=2000,                # Maximum number of training iterations
    random_state=10              # Random seed for reproducibility
)

mlp_regressor_1.fit(x_train_scale, y_train)

CPU times: total: 500 ms
Wall time: 275 ms


MLPRegressor(hidden_layer_sizes=(100, 200, 100, 50), max_iter=2000,
             random_state=10)

In [84]:
# Step 4: Evaluate the model
y_pred = mlp_regressor_1.predict(x_test_scale)
y_pred = [round(pred, 0) for pred in y_pred]

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
score = mlp_regressor_1.score(x_test_scale, y_test)

print(f"Mean Squared Error: {mse}")
print(f"R-squared (R2) Score: {r2}")
print(f"Model Score: {score}")

Mean Squared Error: 2.772727272727273
R-squared (R2) Score: 0.013960323291697385
Model Score: 0.06405561058864184


In [85]:
for i in range(len(y_pred)):
    print(y_pred[i],y_test[i])
    
correct_pred = []
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        correct_pred.append('correct')
print(len(correct_pred)/len(y_pred))

1.0 7.0
2.0 2.0
1.0 0.0
2.0 1.0
2.0 4.0
1.0 0.0
2.0 5.0
1.0 2.0
2.0 1.0
1.0 1.0
1.0 1.0
1.0 0.0
1.0 0.0
2.0 2.0
2.0 1.0
1.0 2.0
1.0 1.0
1.0 2.0
2.0 3.0
1.0 1.0
1.0 2.0
1.0 1.0
0.3181818181818182


#### Away ML Model

In [86]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x[selected_features], Away_score, test_size=0.2, random_state=42)

# Standardize features (scaling)
scaler = StandardScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

In [87]:
%%time
# Step 3: Create and train the MLPRegressor model
#['identity', 'logistic', 'relu', 'softmax', 'tanh']

mlp_regressor_2 = MLPRegressor(
    hidden_layer_sizes=(100, 200, 100, 50),  # Define the architecture of the neural network
    activation="relu",            # Activation function (e.g., "relu", "tanh")
    solver="adam",                # Optimization algorithm
    max_iter=2000,                # Maximum number of training iterations
    random_state=42               # Random seed for reproducibility
)

mlp_regressor_2.fit(x_train_scale, y_train)

CPU times: total: 719 ms
Wall time: 370 ms


MLPRegressor(hidden_layer_sizes=(100, 200, 100, 50), max_iter=2000,
             random_state=42)

In [88]:
# Step 4: Evaluate the model
y_pred = mlp_regressor_2.predict(x_test_scale)
y_pred = [round(pred, 0) for pred in y_pred]

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
score = mlp_regressor_2.score(x_test_scale, y_test)

print(f"Mean Squared Error: {mse}")
print(f"R-squared (R2) Score: {r2}")
print(f"Model Score: {score}")

Mean Squared Error: 1.0
R-squared (R2) Score: -0.0803571428571428
Model Score: -0.08952500627986693


In [89]:
for i in range(len(y_pred)):
    print(abs(y_pred[i]),y_test[i])
    
correct_pred = []
for i in range(len(y_pred)):
    if abs(y_pred[i]) == abs(y_test[i]):
        correct_pred.append('correct')
print(len(correct_pred)/len(y_pred))

1.0 1.0
2.0 2.0
3.0 2.0
2.0 3.0
1.0 3.0
0.0 0.0
0.0 0.0
2.0 2.0
2.0 1.0
2.0 2.0
1.0 2.0
1.0 2.0
1.0 1.0
2.0 2.0
1.0 0.0
2.0 1.0
0.0 1.0
3.0 1.0
1.0 0.0
2.0 0.0
1.0 0.0
2.0 2.0
0.4090909090909091


#### Match Outcome ML Model

In [90]:
%%time
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x[selected_features], Match_Outcome_Label, test_size=0.2, random_state=42)

# Standardize features (scaling)
scaler = StandardScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

# Step 3: Create and train the MLPRegressor model
#['identity', 'logistic', 'relu', 'softmax', 'tanh']

mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(100, 200, 100),  # Define the architecture of the neural network
    activation="relu",            # Activation function (e.g., "relu", "tanh")
    solver="adam",                # Optimization algorithm
    max_iter=1500,                # Maximum number of training iterations
    random_state=42               # Random seed for reproducibility
)

mlp_classifier.fit(x_train_scale, y_train)

CPU times: total: 531 ms
Wall time: 263 ms


MLPClassifier(hidden_layer_sizes=(100, 200, 100), max_iter=1500,
              random_state=42)

In [91]:
# Step 4: Evaluate the model
y_pred = mlp_classifier.predict(x_test_scale)
y_pred = [round(pred, 0) for pred in y_pred]

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
score = mlp_classifier.score(x_test_scale, y_test)

print(f"Mean Squared Error: {mse}")
print(f"R-squared (R2) Score: {r2}")
print(f"Model Score: {score}")

Mean Squared Error: 0.5
R-squared (R2) Score: 0.2065573770491803
Model Score: 0.6363636363636364


In [92]:
for i in range(len(y_pred)):
    print(y_pred[i],y_test[i])
    
correct_pred = []
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        correct_pred.append('correct')
print(len(correct_pred)/len(y_pred))

0 0
1 1
1 1
1 1
1 1
0 0
0 0
2 2
1 2
0 0
0 1
0 1
0 0
0 0
2 0
1 2
0 0
1 2
1 2
1 0
0 0
1 1
0.6363636363636364


In [94]:
import joblib

# Assuming you have a trained model called 'model'
# model = YourTrainedModel()

# Save the model to a pickle file
# joblib.dump(mlp_regressor_1, 'homescore_predictor.pkl')
# joblib.dump(mlp_regressor_2, 'awayscore_predictor.pkl')
# joblib.dump(mlp_classifier, 'outcome_predictor.pkl')

# Load the model from the pickle file
# loaded_model = joblib.load('model.pkl')

# Now, you can use the loaded model to make predictions
# For example, assuming 'X_test' is your test data
# predictions = loaded_model.predict(X_test)